In [1]:
import util
import pandas as pd
import numpy as np
import twsq
from twsq.alpha import Alpha

In [2]:
layer1_coin_names = util.get_layer1_universe()


Fetched page 1 with 250 coins.
Fetched page 2 with 250 coins.
Fetched page 3 with 250 coins.
Fetched page 4 with 3 coins.


In [ ]:
class MRBRLong(Alpha):
    def filter_universe(self, universe, window_needed, freq='1h'):
        """
        Filter the provided universe to include only symbols for which data exists.

        Parameters:
        -----------
        universe : list
            List of asset symbols.
        window_needed : int
            Minimum number of bars required.
        freq : str, optional
            Frequency of the bars (default '1h').

        Returns:
        --------
        list
            Filtered list of symbols.
        """
        valid_symbols = []
        for symbol in universe:
            try:
                bars = self.get_lastn_bars(symbol + '/USD', n=window_needed, freq=freq)
                if not bars.empty and len(bars) >= window_needed:
                    valid_symbols.append(symbol)
            except Exception as e:
                continue
        return valid_symbols
    """
    Mean reversion breakout strategy for short positions.
    """

    def prepare(self, universe=None, enter_zscore = -3, exit_zscore = 1, enter_lookback=12, exit_lookback=15, dollar_exposure=1000):
        """
        Initialize strategy parameters.
        Parameters:
        -----------
        universe : list
            List of asset symbols to consider (e.g. ['BTC/USD', 'ETH/USD']).
        enter_lookback : int, optional
            Lookback period for the entry threshold.
        exit_lookback : int, optional
            Lookback period for the exit threshold.
        dollar_exposure : float, optional
            Total dollar exposure.
        """
        if universe is None:
            universe = util.get_layer1_universe()
        universe = [str(symbol).upper() for symbol in universe]
        self.enter_lookback = enter_lookback
        self.exit_lookback = exit_lookback
        self.dollar_exposure = dollar_exposure
        window_needed = max(self.enter_lookback, self.exit_lookback)
        self.universe = self.filter_universe(universe, window_needed)
        self.current_signals = {symbol: 0 for symbol in universe}
    
    def rebalance(self):
        """
        Compute mr breakout signals and rebalance positions.
        Dollar exposure is total exposure across all short positions.
        """
        target = {}
        signals = {}
        # Determine the number of bars needed from history.
        window_needed = max(self.enter_lookback, self.exit_lookback)
        
        # Compute signals for each asset.
        for symbol in self.universe:
            try:
                bars = self.get_lastn_bars(symbol + "/USD", n=window_needed, freq='1h')
                # Check if we have enough data.
                if bars.empty:
                    signal = 0
                else:
                    price_series = bars
                    ret = price_series['close'].pct_change()
                    enter_min = ret.iloc[-self.enter_lookback:].min()
                    exit_max = ret.iloc[-self.exit_lookback:].max()
                    # enter_max = price_series['close'].rolling(window=self.enter_lookback, min_periods = 1).max().iloc[-1]
                    # exit_min = price_series['close'].rolling(window=self.exit_lookback, min_periods = 1).min().iloc[-1]
                    current_price = ret.iloc[-1]
                    
                    # Determine signal based on the most recent value.
                    if not price_series.empty:
                        if current_price == enter_min:
                            signal = 1  # Enter short
                        elif current_price == exit_max:
                            signal = 0   # Exit short
                        else:
                            signal = self.current_signals[symbol]
                    else:
                        signal = 0
            except Exception as e:
                # In case of error, close position.
                print(f"Error processing {symbol}: {e}")
                signal = 0
            self.current_signals[symbol] = signal
            signals[symbol] = signal
        longs = [symbol for symbol, sig in signals.items() if sig == 1]
        nlong = len(longs)

        # Compute target positions using total dollar exposure split across short positions.
        for symbol, signal in signals.items():
            try:
                price = self.get_current_price(symbol + '/USD')
            except Exception as e:
                continue
            
            if signal == 1 and nlong > 0:
                # print(f"Shorting {symbol} at price {price}")
                # Allocate total exposure equally among all assets with a short signal.
                target[symbol] = (self.dollar_exposure / nlong) / price
            else:
                target[symbol] = 0
            
        # Rebalance portfolio to target positions using TWSQ's built-in function.
        self.trade_to_target(target, quote='USD', route=True)

